In [1]:
'''
TESTING OUT QUERIES FOR SUB TASK A
'''
print()

### Importing required libraries

In [1]:
import sqlite3
import numpy as np
import pandas as pd

### Global variables

In [2]:
DATABASE = '../datasets/climate.db'

### Landing page stats

In [ ]:
with sqlite3.connect(DATABASE) as con:
    most_weather_stations_query = (
        """
        select state_name
        from states
        where state_id = (
            select state_id
            from weather_stations
            group by state_id
            order by count(site_id) desc
            limit 1
        );
        """
    )
    
    state = pd.read_sql_query(most_weather_stations_query,con).iloc[0,"state_name"]

state

'QLD'

In [16]:
with sqlite3.connect(DATABASE) as con:
    hottest_year_query = (
        """
            select strftime('%Y',DMY) as year, AVG(MaxTemp) as avg_max_temp
            from bom_data
            group by strftime('%Y',DMY) 
            order by 2 desc
            limit 1;
        """
    )

    hottest_year = pd.read_sql_query(hottest_year_query, con).loc[0,"year"]

hottest_year

'2019'

In [7]:
with sqlite3.connect(DATABASE) as con:
    temp = pd.read_sql_query('select max(MaxTemp) as highest_temp_recorded from bom_data', con)

float(temp.loc[0,'highest_temp_recorded'])

50.5

In [20]:
with sqlite3.connect(DATABASE) as con:
    driest_year_query = (
        """
            select strftime('%Y',DMY) as year, SUM(Precipitation) as total_precipitation
            from bom_data
            group by strftime('%Y',DMY) 
            order by 2 asc
            limit 1;
        """
    )

    driest_year_table = pd.read_sql_query(driest_year_query, con)

driest_year_table

,year,total_precipitation
0,2019,67730.5


### Shallow Glance

In [ ]:
with sqlite3.connect(DATABASE) as con: # helps with setting start and end lat bounds when a state is selected
    min_max_lat = pd.read_sql_query(
        '''
        select MIN(lat), ROUND(MIN(lat)-0.01,2), MAX(lat), ROUND(MAX(lat)+0.01,2)
        from weather_stations
        where state_id = 2;
        '''
, con)

min_max_lat

,MIN(lat),"ROUND(MIN(lat)-0.01,2)",MAX(lat),"ROUND(MAX(lat)+0.01,2)"
0,-29.0389,-29.05,-12.1892,-12.18


In [13]:
with sqlite3.connect(DATABASE) as con:
    all_min_max_lat = pd.read_sql_query(
        '''
        select state_name, ROUND(MIN(lat)-0.01,2) as min_lat, ROUND(MAX(lat)+0.01,2) as max_lat
        from weather_stations ws join states s
        on ws.state_id = s.state_id
        group by s.state_name;
        '''
        ,con)
        
all_min_max_lat.set_index('state_name').to_dict(orient='records')

[{'min_lat': -68.58, 'max_lat': -54.49},
 {'min_lat': -29.05, 'max_lat': -12.18},
 {'min_lat': -36.5, 'max_lat': -28.63},
 {'min_lat': -23.81, 'max_lat': -11.39},
 {'min_lat': -28.63, 'max_lat': -12.78},
 {'min_lat': -37.76, 'max_lat': -27.55},
 {'min_lat': -43.67, 'max_lat': -40.08},
 {'min_lat': -39.14, 'max_lat': -34.23},
 {'min_lat': -34.38, 'max_lat': -13.74}]

In [21]:
with sqlite3.connect(DATABASE) as con:
    states_df = pd.read_sql_query('''select state_name from states''',con)

list(states_df.loc[:,'state_name'].values)

['A.A.T.', 'A.E.T.', 'N.S.W.', 'N.T.', 'QLD', 'S.A.', 'TAS', 'VIC', 'W.A.']

In [3]:
with sqlite3.connect(DATABASE) as con:
    temp = pd.read_sql_query(
        '''
        select s.state_name, COUNT(ws.site_id) as weather_stations
        from weather_stations ws join states s
        on ws.state_id = s.state_id
        group by s.state_name
        order by weather_stations desc;
        '''
        ,con)

temp.to_html(classes='data-table')

'<table border="1" class="dataframe data-table">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>state_name</th>\n      <th>weather_stations</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>QLD</td>\n      <td>28</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>W.A.</td>\n      <td>27</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>N.S.W.</td>\n      <td>22</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>VIC</td>\n      <td>21</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>N.T.</td>\n      <td>14</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>TAS</td>\n      <td>12</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>S.A.</td>\n      <td>12</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>A.E.T.</td>\n      <td>3</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td>A.A.T.</td>\n      <td>2</td>\n    </tr>\n  </tbody>\n</table>'

In [ ]:
np.concatenate([np.arange(temp.shape[0]).reshape(-1,1),temp.values],axis=1).tolist() # sending query data to flask as a list of list

[[0, 'QLD', 28],
 [1, 'W.A.', 27],
 [2, 'N.S.W.', 22],
 [3, 'VIC', 21],
 [4, 'N.T.', 14],
 [5, 'TAS', 12],
 [6, 'S.A.', 12],
 [7, 'A.E.T.', 3],
 [8, 'A.A.T.', 2]]

In [22]:
list(temp.columns)

['state_name', 'weather_stations']

In [9]:
with sqlite3.connect(DATABASE) as con:
    query = (
        '''
select 
    site_id as site,
    name,
    ROUND(lat, 2) as lat,
    ROUND(long,2) as long,
    REPLACE(state_name,'.','') as state,
    region_name as region
from
    weather_stations ws 
        LEFT JOIN states s on ws.state_id = s.state_id
        LEFT JOIN regions r on ws.region_id = r.region_id
'''
)
    tb = pd.read_sql_query(query,con)

tb

,site,name,lat,long,state,region
0,1006,WYNDHAM AERO,-15.51,128.15,WA,Wyndham East Kimberley
1,1007,TROUGHTON ISLAND,-13.75,126.15,WA,Kimberley
2,3003,BROOME AIRPORT,-17.95,122.24,WA,Broome
3,3032,DERBY AERO,-17.37,123.66,WA,Derby West Kimberley
4,4019,MANDORA,-19.74,120.84,WA,Broome
...,...,...,...,...,...,...
136,200283,WILLIS ISLAND,-16.29,149.97,AET,N.A.
137,200284,COCOS ISLAND AIRPORT,-12.19,96.83,AET,N.A.
138,200288,NORFOLK ISLAND AERO,-29.04,167.94,AET,N.A.
139,300000,DAVIS,-68.57,77.97,AAT,N.A.


In [25]:
with sqlite3.connect(DATABASE) as con:
    df = pd.read_sql_query(
        '''
select DISTINCT(Location) as missing_stations
from bom_data bd
    LEFT JOIN weather_stations ws on bd.Location = ws.site_id
where ws.site_id IS NULL
''',con)

df

,missing_stations
0,5017
1,36031
2,58077


Observations:
- The above 3 stations are present in the ```bom_data``` but not included in the ```weather_stations``` table, we will be excluding these stations. 

In [59]:
with sqlite3.connect(DATABASE) as con:
    df = pd.read_sql_query(
        '''
select DISTINCT(site_id) as missing_stations
from bom_data bd
    RIGHT JOIN weather_stations ws on bd.Location = ws.site_id
where bd.Location IS NULL
''',con)

df

,missing_stations
0,1006
1,9240
2,9542
3,10692
4,14932
5,18106
6,18116
7,18120
8,22803
9,23000


Observations:
- The above stations are present in the ```weather_stations``` but not included in the ```bom_data``` table, we will be excluding these stations. 

In [40]:
with sqlite3.connect(DATABASE) as con:
        query = (
            '''
            select
                state,
                ROUND(AVG(avgEvaporation),2) as 'avg annual evaporation (mm)'
            from (
                select
                    strftime('%Y',DMY) as year,
                    REPLACE(s.state_name,".","") as state,
                    AVG(Evaporation) as avgEvaporation
                from bom_data bd
                    LEFT JOIN weather_stations ws on bd.Location = ws.site_id
                    LEFT JOIN states s on ws.state_id = s.state_id
                where ws.site_id IS NOT NULL
                group by s.state_name, strftime('%Y',DMY)
            )
            where avgEvaporation IS NOT NULL
            group by state;
            
    '''
    )
        df = pd.read_sql_query(query,con)


In [41]:
df

,state,avg annual evaporation (mm)
0,AET,5.41
1,NSW,5.27
2,NT,8.19
3,QLD,6.50
4,SA,8.13
5,TAS,3.05
6,VIC,5.06
7,WA,7.43


In [47]:
def table1(state, start_lat, end_lat):

    query = f'''
    SELECT 
        ws.site_id as site,
        ws.name as name,
        r.region_name as region,
        ws.lat as latitude
    FROM weather_stations ws 
        LEFT JOIN states s on ws.state_id = s.state_id
        LEFT JOIN regions r on ws.region_id = r.region_id
    WHERE 
        (s.state_name  = "{state}")
        AND 
        (ws.lat BETWEEN {start_lat} AND {end_lat})'''
    
    with sqlite3.connect(DATABASE) as con:
        df = pd.read_sql_query(query,con)
    
    return df, query

In [50]:
df, query = table1("QLD",-28.64,-12.77)

In [51]:
df

,site,name,region,latitude
0,28004,PALMERVILLE,Cook,-15.9999
1,28008,LOCKHART RIVER AIRPORT,Cook,-12.7850
2,29038,KOWANYAMA AIRPORT,Aboriginal Shire of Kowanyama,-15.4818
3,29127,MOUNT ISA AERO,Mount Isa,-20.6778
4,31011,CAIRNS AERO,Cairns,-16.8736
5,31037,LOW ISLES LIGHTHOUSE,Douglas,-16.3842
6,32037,SOUTH JOHNSTONE EXP STN,Cassowary Coast,-17.6053
7,32040,TOWNSVILLE AERO,Townsville,-19.2483
8,33002,AYR DPI RESEARCH STN,Burdekin,-19.6169
9,33045,MACKAY AERO,Mackay,-21.1706


In [54]:
def table2(state, start_lat, end_lat, metric, period):

    # Precipitation
    # MaxTemp
    # MinTemp
    # Evaporation
    # Sunshine


    periods = { 'January':'01',
    'February':'02',
    'March':'03',
    'April':'04',
    'May':'05',
    'June':'06',
    'July':'07',
    'August':'08',
    'September':'09',
    'October':'10',
    'November':'11',
    'December':'12'}
   
    metricNames = {'Precipitation':['avgRainfall','avg rainfall (mm)'],'MaxTemp':['avgMaxTemp','avg max temp (℃)'],'MinTemp':['avgMinTemp','avg min temp (℃)'],'Evaporation':['avgEvaporation','avg evaporation (mm)'],'Sunshine':['avgSunshine','avg sunshine (hrs)']}

    if period == 'Annual':
        query = f'''

        WITH stations_within_area AS (
                SELECT 
                    ws.site_id as site,
                    r.region_name as region,
                FROM weather_stations ws 
                    LEFT JOIN states s on ws.state_id = s.state_id
                    LEFT JOIN regions r on ws.region_id = r.region_id
                WHERE 
                    (s.state_name  = "{state}")
                    AND 
                    (ws.lat BETWEEN {start_lat} AND {end_lat})
        )
        WITH stations_per_region AS (
                SELECT
                    region,
                    COUNT(*) as '# weather stations'
                FROM stations_within_area swa
                GROUP BY swa.region
        )
                SELECT
                    region,
                    ROUND(AVG({metricNames[metric][0]}),2) as '{metricNames[metric][1]}'
                FROM (
                    SELECT
                        strftime('%Y',DMY) as year,
                        swa.region,
                        AVG({metric}) as {metricNames[metric][0]}
                    FROM bom_data bd
                        LEFT JOIN stations_within_area swa on bd.Location = swa.site
                    WHERE swa.site IS NOT NULL
                    GROUP BY swa.region, strftime('%Y',DMY)
                )
                WHERE {metricNames[metric][0]} IS NOT NULL
                GROUP BY region;
            '''
    else:
        pass
    
    with sqlite3.connect(DATABASE) as con:
        df = pd.read_sql_query(query,con)
    
    return df, query

### Deep Dive

In [14]:
def get_stations_in_state(state):
    
    with sqlite3.connect(DATABASE) as con:
        state_id = pd.read_sql_query(
            f'''
            select state_id
            from states
            where state_name = "{state}";
            '''
            , con).loc[0, "state_id"]
        
        stations_df = pd.read_sql_query(
            f'''
            select site_id, name
            from weather_stations
            where state_id = {state_id};
            '''
            , con)
        
        return stations_df.to_dict(orient='records')

In [15]:
get_stations_in_state("A.A.T.")

[{'site_id': 300000, 'name': 'DAVIS'},
 {'site_id': 300004, 'name': 'MACQUARIE ISLAND'}]

In [38]:
def get_decades():

    with sqlite3.connect(DATABASE) as con:
        years = pd.read_sql_query(
            '''
select DISTINCT(FLOOR(CAST(strftime("%Y",DMY) as INT) / 10) * 10) as decades
from bom_data
'''
,con)
        
    return years.loc[:,"decades"].values

In [39]:
get_decades()

array([1970, 1980, 1990, 2000, 2010, 2020])

In [112]:
def get_similar_stations(metric, period_1, period_2, reference_station, topN = None):

    metricNames = {'Precipitation':'avg rainfall (mm)','MaxTemp':'avg max temp (℃)','MinTemp':'avg min temp (℃)'}

    period_1_end = int(period_1) + 9
    period_2_end = int(period_2) + 9
    metric_alias = metricNames[metric]

    with sqlite3.connect(DATABASE) as con:

        query = f'''
        with period1_stats as (
            select Location, AVG({metric}) as "{metric_alias} {period_1}'s"
            from bom_data
            where CAST(strftime('%Y', DMY) as INT) BETWEEN {period_1} AND {period_1_end}
            group by Location
        ),
        period2_stats as (
            select Location, AVG({metric}) as "{metric_alias} {period_2}'s"
            from bom_data
            where CAST(strftime('%Y', DMY) as INT) BETWEEN {period_1} AND {period_2_end}
            group by Location
        ),
        combined_stats as (
            select 
                p1.Location,
                "{metric_alias} {period_1}'s",
                "{metric_alias} {period_2}'s",
                100 * ("{metric_alias} {period_2}'s" - "{metric_alias} {period_1}'s") / "{metric_alias} {period_1}'s" as "% change"
            from period1_stats p1
                INNER JOIN period2_stats p2 on p1.Location = p2.Location
            WHERE "{metric_alias} {period_1}'s" IS NOT NULL AND 
                "{metric_alias} {period_2}'s" IS NOT NULL
        ),
        final_cte as (
            select 
                *,
                "% change" - (select "% change" from combined_stats where Location = {reference_station}) as "relative difference",
                ABS("% change" - (select "% change" from combined_stats where Location = {reference_station})) as "absDiff"
            from combined_stats
            order by absDiff
        )
        select
            name as "station name",
            "{metric_alias} {period_1}'s",
            "{metric_alias} {period_2}'s",
            "% change",
            "relative difference"
        from final_cte fc
            LEFT JOIN weather_stations ws on fc.Location = ws.site_id
        where "station name" IS NOT NULL
        order by absDiff
        limit {topN + 1}
    '''
        df = pd.read_sql_query(query, con)

    return df
   

In [115]:
get_similar_stations("MaxTemp", "1970", "2000", "66137", 10)

,station name,avg max temp (℃) 1970's,avg max temp (℃) 2000's,% change,relative difference
0,BANKSTOWN AIRPORT AWS,22.977492,23.134473,0.683194,0.000000
1,DERBY AERO,34.263682,34.496659,0.679955,-0.003239
2,SALMON GUMS RES.STN.,23.246827,23.408607,0.695923,0.012730
3,MARDIE,33.792620,34.029127,0.699877,0.016683
4,CAPE OTWAY LIGHTHOUSE,16.772115,16.892959,0.720505,0.037311
...,...,...,...,...,...
6,MEEKATHARRA AIRPORT,28.853204,29.035742,0.632646,-0.050548
7,KALGOORLIE-BOULDER AIRPORT,25.177979,25.332939,0.615458,-0.067735
8,EMU CREEK STATION,33.417521,33.668887,0.752199,0.069005
9,LOCKHART RIVER AIRPORT,29.643554,29.825100,0.612428,-0.070765


Observations:

- Lots of stations don't have value for Evaporation, hence it was excluded from analysis. Better data preparation is advised for future improvements